In [2]:
#!pip install transformers datasets      
import sys
sys.path.insert(0,'f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/')
#sys.path.insert(0,'C:/Users/Mey/Documents/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/')

#!pip install transformers datasets      
import sys
sys.path.insert(0,'f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/')
#sys.path.insert(0,'C:/Users/Mey/Documents/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/')

from PIL import Image
import torch
from torch.utils.data import Dataset
from transformers import ViTImageProcessor, ViTForImageClassification
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import Trainer, TrainingArguments
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# بارگذاری feature extractor از ViT
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

# تعریف transform مناسب برای پردازش تصاویر
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # تغییر اندازه به 224x224
    transforms.ToTensor(),          # تبدیل تصویر به Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # نرمال‌سازی
])

# ساخت یک Dataset سفارشی
class CustomImageDataset(Dataset):
    def __init__(self, data_path, transform=None):
        self.data = datasets.ImageFolder(root=data_path, transform=transform)
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image, label = self.data[idx]
        # تبدیل تصویر به PIL.Image و سپس پردازش با ViTImageProcessor
        image = transforms.ToPILImage()(image)  # تبدیل از Tensor به PIL.Image
        pixel_values = processor(images=image, return_tensors="pt").pixel_values.squeeze(0)
        return {"pixel_values": pixel_values, "labels": label}
    
path_data = 'f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/dataset'
# بارگذاری داده‌ها با استفاده از CustomImageDataset
train_dataset = CustomImageDataset(path_data + '/train/', transform=train_transform)
test_dataset = CustomImageDataset(path_data + '/test/', transform=train_transform)

# بارگذاری مدل ViT برای طبقه‌بندی تصویر
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=16)  # تغییر تعداد کلاس‌ها به 16

# تنظیمات آموزش
training_args = TrainingArguments(
    output_dir="./vit_finetuned",         # مکانی برای ذخیره مدل نهایی
    evaluation_strategy="epoch",          # ارزیابی در پایان هر epoch
    learning_rate=2e-5,                   # نرخ یادگیری برای fine-tuning
    per_device_train_batch_size=16,       # اندازه بچ برای آموزش
    per_device_eval_batch_size=64,        # اندازه بچ برای ارزیابی
    num_train_epochs=100,                   # تعداد epochs برای آموزش
    weight_decay=0.01,                    # تنظیمات weight decay
    save_total_limit=2,                   # ذخیره تنها آخرین ۲ checkpoint
    logging_dir='./logs',                 # مسیر ذخیره لاگ‌ها
)

# تابع محاسبه متریک‌ها
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

# راه‌اندازی Trainer
trainer = Trainer(
    model=model,                           # مدل
    args=training_args,                    # تنظیمات آموزش
    train_dataset=train_dataset,           # داده‌های آموزشی
    eval_dataset=test_dataset,             # داده‌های ارزیابی
    compute_metrics=compute_metrics        # محاسبه متریک‌ها
)

# شروع آموزش مدل
trainer.train()

# ارزیابی مدل پس از آموزش
results = trainer.evaluate()

print("Evaluation Results:")
print(results)

# استخراج ویژگی‌ها و برچسب‌ها برای t-SNE
model.eval()
train_features = []
train_labels = []

# استخراج ویژگی‌ها از مدل
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=False)

with torch.no_grad():
    for batch in train_loader:
        inputs = batch["pixel_values"]
        labels = batch["labels"]
        outputs = model(pixel_values=inputs)  # ورودی‌های پیش پردازش شده به مدل می‌دهیم
        features = outputs.logits.cpu().numpy()
        train_features.extend(features)
        train_labels.extend(labels.cpu().numpy())

train_features = np.array(train_features)
train_labels = np.array(train_labels)

# استفاده از t-SNE برای کاهش ابعاد
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(train_features)

# رسم نتیجه t-SNE
plt.figure(figsize=(10, 8))
scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=train_labels, cmap='viridis', s=10)
plt.colorbar(scatter)
plt.title("t-SNE Visualization of ViT Features")
plt.show()

# ذخیره مدل آموزش‌دیده
trainer.save_model("f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/vit_finetuned_model")




###################################################################################################################



# import torch
# from torch.utils.data import DataLoader
# from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer, TrainingArguments
# from datasets import load_dataset
# from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt
# import numpy as np

# # Load CIFAR-10 dataset from Hugging Face
# dataset = load_dataset("cifar10")

# # Split the dataset into train and validation sets
# train_dataset = dataset['train']
# val_dataset = dataset['test']

# # Load pre-trained ViT feature extractor
# feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

# # Preprocessing function for ViT
# def preprocess_function(examples):
#     return feature_extractor([x['img'] for x in examples], return_tensors="pt")

# # Preprocess the dataset
# train_dataset = train_dataset.map(preprocess_function, batched=True)
# val_dataset = val_dataset.map(preprocess_function, batched=True)

# # Set format for PyTorch
# train_dataset.set_format(type='torch', columns=['input_ids', 'label'])
# val_dataset.set_format(type='torch', columns=['input_ids', 'label'])

# # Load pre-trained ViT model for image classification
# model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=10)

# # Define the training arguments
# training_args = TrainingArguments(
#     output_dir="./vit_finetuned",         # Where to store the final model
#     evaluation_strategy="epoch",          # Evaluate at the end of each epoch
#     learning_rate=2e-5,                   # Learning rate for fine-tuning
#     per_device_train_batch_size=16,       # Batch size for training
#     per_device_eval_batch_size=64,        # Batch size for evaluation
#     num_train_epochs=3,                   # Number of training epochs
#     weight_decay=0.01,                    # Weight decay for regularization
#     save_total_limit=2,                   # Save only the last 2 checkpoints
#     logging_dir='./logs',                 # Directory for logs
# )

# # Define the metrics for evaluation
# def compute_metrics(p):
#     preds = np.argmax(p.predictions, axis=1)
#     labels = p.label_ids
#     acc = accuracy_score(labels, preds)
#     f1 = f1_score(labels, preds, average='weighted')
#     precision = precision_score(labels, preds, average='weighted')
#     recall = recall_score(labels, preds, average='weighted')

#     return {
#         "accuracy": acc,
#         "f1": f1,
#         "precision": precision,
#         "recall": recall
#     }

# # Initialize the Trainer
# trainer = Trainer(
#     model=model,                           # The model to be fine-tuned
#     args=training_args,                    # Training arguments
#     train_dataset=train_dataset,           # Training dataset
#     eval_dataset=val_dataset,              # Validation dataset
#     compute_metrics=compute_metrics        # Evaluation metrics
# )

# # Start fine-tuning
# trainer.train()

# # Evaluate the model after fine-tuning
# results = trainer.evaluate()
# print("Evaluation Results:")
# print(results)

# # Get features and labels for t-SNE
# model.eval()
# train_features = []
# train_labels = []

# # Extract features from the model
# with torch.no_grad():
#     for batch in DataLoader(train_dataset, batch_size=32):
#         inputs = batch['input_ids']
#         labels = batch['label']
#         outputs = model(**inputs)
#         features = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
#         train_features.extend(features)
#         train_labels.extend(labels.cpu().numpy())

# train_features = np.array(train_features)
# train_labels = np.array(train_labels)

# # Use t-SNE for dimensionality reduction
# tsne = TSNE(n_components=2, random_state=42)
# tsne_results = tsne.fit_transform(train_features)

# # Plot the t-SNE result
# plt.figure(figsize=(10, 8))
# scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=train_labels, cmap='viridis', s=10)
# plt.colorbar(scatter)
# plt.title("t-SNE Visualization of ViT Features")
# plt.show()

# # Save the fine-tuned model
# trainer.save_model("./vit_finetuned_model")



Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\pc\anaconda3\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,2.703568,0.189229,0.126547,0.196929,0.189229
2,No log,2.629801,0.283115,0.216089,0.389690,0.283115
3,No log,2.575210,0.331878,0.278573,0.420165,0.331878
4,No log,2.512361,0.405871,0.372301,0.454413,0.405871
5,No log,2.464804,0.437894,0.409648,0.463512,0.437894
6,No log,2.413861,0.467977,0.441855,0.489784,0.467977
7,No log,2.372236,0.476468,0.450578,0.505219,0.476468
8,No log,2.339723,0.488355,0.467950,0.521171,0.488355
9,No log,2.299177,0.485929,0.457268,0.522016,0.485929
10,No log,2.258552,0.505095,0.479813,0.538071,0.505095


Evaluation Results:
{'eval_loss': 1.546509861946106, 'eval_accuracy': 0.636098981077147, 'eval_f1': 0.6343566647344295, 'eval_precision': 0.6553597648224023, 'eval_recall': 0.636098981077147, 'eval_runtime': 25.2495, 'eval_samples_per_second': 163.251, 'eval_steps_per_second': 2.574, 'epoch': 100.0}


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [12]:
# !pip install --upgrade transformers[torch]
# !pip install --upgrade torch
#!pip show tokenizers





In [8]:
!pip uninstall tokenizers
!pip install tokenizers==0.19.1


  Using cached tokenizers-0.19.1-cp39-none-win_amd64.whl (2.2 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.46.3 requires tokenizers<0.21,>=0.20, but you have tokenizers 0.19.1 which is incompatible.


In [9]:
!pip install --upgrade transformers
!pip show tokenizers

  Using cached tokenizers-0.20.3-cp39-none-win_amd64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
Name: tokenizers
Version: 0.20.3
Summary: 
Home-page: 
Author: Anthony MOI <m.anthony.moi@gmail.com>
Author-email: Nicolas Patry <patry.nicolas@protonmail.com>, Anthony Moi <anthony@huggingface.co>
License: 
Location: c:\programdata\anaconda3\lib\site-packages
Requires: huggingface-hub
Required-by: transformers


In [11]:
!pip uninstall transformers


^C


In [22]:
!pip show tokenizers



^C


In [23]:
!pip install transformers==4.28.0

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\programdata\\anaconda3\\lib\\site-packages\\tokenizers-0.13.3.dist-info\\METADATA'



In [2]:
pip uninstall tokenizers


^C
Note: you may need to restart the kernel to use updated packages.


In [25]:
!pip install tokenizers==0.19.1

^C


In [1]:
!pip show tokenizers

Name: tokenizers
Version: 0.19.1
Summary: 
Home-page: 
Author: Anthony MOI <m.anthony.moi@gmail.com>
Author-email: Nicolas Patry <patry.nicolas@protonmail.com>, Anthony Moi <anthony@huggingface.co>
License: 
Location: c:\programdata\anaconda3\lib\site-packages
Requires: huggingface-hub
Required-by: transformers


In [3]:
pip install transformers==4.28.0

  Using cached tokenizers-0.13.3-cp39-cp39-win_amd64.whl (3.5 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install tokenizers==0.13.3

In [5]:
!pip show tokenizers


Name: tokenizers
Version: 0.13.3
Summary: Fast and Customizable Tokenizers
Home-page: https://github.com/huggingface/tokenizers
Author: Anthony MOI
Author-email: anthony@huggingface.co
License: Apache License 2.0
Location: c:\programdata\anaconda3\lib\site-packages
Requires: 
Required-by: transformers


In [5]:
pip install accelerate>={ACCELERATE_MIN_VERSION}

Note: you may need to restart the kernel to use updated packages.
